In [1]:
import os
import boto3
import pandas as pd
import json
import psycopg2
import sqlalchemy as sa
import json

In [3]:
credfile = 'redshift_creds_template.json.nogit'

# Read credentials to a dictionary
with open(credfile) as fh:
    creds = json.loads(fh.read())

def get_conn(creds): 
    conn = psycopg2.connect(dbname=creds['db_name'], 
                            user=creds['username'], 
                            password=creds['password'],
                            port=creds['port_num'],
                            host=creds['host_name'])
    return conn

def get_df(creds, query):
    with get_conn(creds) as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            result_set = cur.fetchall()
            colnames = [desc.name for desc in cur.description]
            df = pd.DataFrame.from_records(result_set, columns=colnames)
    return df

In [4]:
import os
import boto3
region = os.getenv('AWS_DEFAULT_REGION')
s3 = boto3.client('s3', region_name=region)

s3_bucket = 'sungardas-aws-services-billing-graphimagebucket-lv7d7riqedp7'

In [5]:
import datetime
from dateutil.relativedelta import relativedelta

import pickle

current_date = datetime.datetime.today()

from_date = datetime.datetime(2016, 8, 1)
to_date = from_date + relativedelta(months=1)

while to_date < current_date:

    sql = "select substring(cast(lineitem_usageenddate as datetime) from 1 for 10) enddatetime, lineItem_UsageAccountId, lineitem_productcode,"
    #sql += " product_servicecode, lineitem_operation, lineitem_usagetype,"
    sql += " sum(lineItem_UsageAmount) usage_amount,"
    sql += " to_char(sum(cast(lineItem_BlendedCost as float)), 'FM999,999,999,990D00') blended,"
    sql += " to_char(sum(cast(lineitem_unblendedcost as float)), 'FM999,999,999,990D00') unblended"
    sql += " from AWSBilling%s" % (from_date.strftime('%Y%m'))
    sql += " where lineitem_lineitemtype != 'Tax'"
    #sql += " and lineitem_usageaccountid = '%s'" %(account_id)
    sql += " and lineitem_usageenddate >= '%s'" % (from_date.strftime('%Y-%m-%d'))
    sql += " and lineitem_usageenddate < '%s'" % (to_date.strftime('%Y-%m-%d'))
    sql += " group by enddatetime, lineItem_UsageAccountId, lineitem_productcode"
    #sql += ", product_servicecode, lineitem_operation, lineitem_usagetype"
    sql += " order by enddatetime, lineItem_UsageAccountId, lineitem_productcode"
    print(sql)
    bill_df = get_df(creds, sql)
    #bill_df
    bill_obj = pickle.dumps(bill_df)
    s3.put_object(Body=bill_obj, Bucket=s3_bucket, Key="dataframes/%s_by_day.pkl" % (from_date.strftime('%Y%m')))
    print("{} data was saved to s3\n".format(from_date.strftime('%Y%m')))
    
    from_date = from_date + relativedelta(months=1)
    to_date = from_date + relativedelta(months=1)

print("All data has been exported to s3")

select substring(cast(lineitem_usageenddate as datetime) from 1 for 10) enddatetime, lineItem_UsageAccountId, lineitem_productcode, sum(lineItem_UsageAmount) usage_amount, to_char(sum(cast(lineItem_BlendedCost as float)), 'FM999,999,999,990D00') blended, to_char(sum(cast(lineitem_unblendedcost as float)), 'FM999,999,999,990D00') unblended from AWSBilling201608 where lineitem_lineitemtype != 'Tax' and lineitem_usageenddate >= '2016-08-01' and lineitem_usageenddate < '2016-09-01' group by enddatetime, lineItem_UsageAccountId, lineitem_productcode order by enddatetime, lineItem_UsageAccountId, lineitem_productcode
201608 data was saved to s3

select substring(cast(lineitem_usageenddate as datetime) from 1 for 10) enddatetime, lineItem_UsageAccountId, lineitem_productcode, sum(lineItem_UsageAmount) usage_amount, to_char(sum(cast(lineItem_BlendedCost as float)), 'FM999,999,999,990D00') blended, to_char(sum(cast(lineitem_unblendedcost as float)), 'FM999,999,999,990D00') unblended from AWSBil

201708 data was saved to s3

select substring(cast(lineitem_usageenddate as datetime) from 1 for 10) enddatetime, lineItem_UsageAccountId, lineitem_productcode, sum(lineItem_UsageAmount) usage_amount, to_char(sum(cast(lineItem_BlendedCost as float)), 'FM999,999,999,990D00') blended, to_char(sum(cast(lineitem_unblendedcost as float)), 'FM999,999,999,990D00') unblended from AWSBilling201709 where lineitem_lineitemtype != 'Tax' and lineitem_usageenddate >= '2017-09-01' and lineitem_usageenddate < '2017-10-01' group by enddatetime, lineItem_UsageAccountId, lineitem_productcode order by enddatetime, lineItem_UsageAccountId, lineitem_productcode
201709 data was saved to s3

select substring(cast(lineitem_usageenddate as datetime) from 1 for 10) enddatetime, lineItem_UsageAccountId, lineitem_productcode, sum(lineItem_UsageAmount) usage_amount, to_char(sum(cast(lineItem_BlendedCost as float)), 'FM999,999,999,990D00') blended, to_char(sum(cast(lineitem_unblendedcost as float)), 'FM999,999,999,9